In [31]:
import pandas as pd
import os
import fnmatch
import numpy as np
from library import start
from library import clean_tea

In [32]:
start.data_path

'/Users/kylieleblancKylie/domino/dofis/data/'

In [33]:
year = 'yr1718'

# Certifications

In [34]:
folder = 'certification_' + year + '/'
teacher_datapath = os.path.join(start.data_path, 'tea', 'teachers', folder)

pattern = "CERTIFICATION*.csv"
if year == 'yr1213' or year == 'yr1314':
    pattern = "CERTIFICATION*.TXT"
cert_files = []
for entry in os.listdir(teacher_datapath):
    if fnmatch.fnmatch(entry, pattern):
        cert_files.append(entry)
cert_files.sort()
dirs_cert = [teacher_datapath + file for file in cert_files]
df_list = [pd.read_csv(file, sep=",", encoding="ISO-8859-1", dtype=object) for file in dirs_cert]
certification = pd.concat(df_list)
certification.head()

,PERSONID_SCRAM,DISTRICT,REGION,CERTIFICATE_ID_NUMBER,CERTIFICATE_LIFE,CREDENTIAL_TYPE,CERTIFICATE_PREPARATION_ROUTE,CERTIFICATE_EFFECTIVE_DATE,CERTIFICATE_EXPIRATION_DATE,SUBJECT_AREA,...,CERTIFICATION_LEVEL,ROLE_CREDENTIALED_FOR,POPULATION_CREDENTIALED_FOR,PREPARATION_PROGRAM,EMAIL_ADDRESS,MAILING_ADDRESS,CITY,ST,ZIP_CD,PHONE
0,*30*5P341,031901,01,2021478,LIFE,Paraprofessional,Paraprofessional Program,06OCT1994:00:00:00,NaN,Other,...,Not Applicable,Educational Aide,Not Applicable,BROWNSVILLE ISD,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
1,*30*5P341,031901,01,2502767,SYR,Emergency Non-Certified,Permit Program,09JAN2002:00:00:00,31AUG2002:00:00:00,Special Education,...,Special Education,Teacher,Special Education,BROWNSVILLE ISD,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
2,*30*5P341,031901,01,2595245,1YR,Probationary,Alternative Program,21AUG2002:00:00:00,21AUG2003:00:00:00,Special Education,...,Special Education,Teacher,Special Education,REG I EDUCATION SERVICE CENTER,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
3,*30*5P341,031901,01,2744684,1YR,Probationary Extension,Alternative Program,31AUG2003:00:00:00,31AUG2004:00:00:00,Special Education,...,Special Education,Teacher,Special Education,REG I EDUCATION SERVICE CENTER,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
4,*30*5P341,031901,01,2922393,5YR,Standard,Alternative Program,02APR2005:00:00:00,31OCT2010:00:00:00,Special Education,...,Special Education,Teacher,Regular Students,REG I EDUCATION SERVICE CENTER,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966


In [35]:
vars_to_keep = {'PERSONID_SCRAM': 'teacher_id', 'DISTRICT': 'district',
                'CREDENTIAL_TYPE': 'cert_type', 'CERTIFICATE_PREPARATION_ROUTE': 'cert_route',
                'CERTIFICATION_LEVEL': 'cert_level', 'CREDENTIALED_GRADES': 'cert_grades',
                'SUBJECT_AREA': 'cert_area', 'SUBJECT': 'cert_subject'}
certification = clean_tea.filter_and_rename_cols(certification, vars_to_keep)
certification.head()

In [36]:
cert_types = {'Emergency Non-Certified': False, 'Emergency Certified': False, 
              'Emergency': False, 'Emergency Teaching': False,
              'Temporary Exemption': False, 'Temporary Teaching Certificate': False,
              'Unknown Permit': False, 'Unknown': False,
              'Special Assignment': False,
              'Paraprofessional': False, 'Standard Paraprofessional': False,
              'Standard': True, 'Provisional': True,
              'Probationary': True, 'Probationary Extension': True, 'Probationary Second Extension': True,
              'Non-renewable': True, 'One Year': True,
              'Visiting International Teacher': True,
              'Professional': True, 'Standard Professional':True}
certification['certification'] = certification['cert_type'].map(cert_types)
certification['vocational'] = np.where((certification['cert_type'] == "Vocational"), True, False)
certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject,certification,vocational
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable,False,False
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education,False,False
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education,True,False


In [37]:
grades_elem = {'Grades EC-4': True, 'Grades EC-6': True,'Grades EC-12': True,
               'Grades PK-KG': True, 'Grades PK-3': True, 'Grades PK-5': True,'Grades PK-12': True,
               'Grades 1-8': True,'Grades 1-6': True,
               'Grades 4-8': True,
               'Grades 6-8': False,'Grades 6-10': False, 'Grades 6-12': False,
               'Grades 7-12': False,
               'Grades 8-12': False}
grades_middle = {'Grades EC-4': False, 'Grades EC-6': True,'Grades EC-12': True,
               'Grades PK-KG': False, 'Grades PK-3': False, 'Grades PK-5': False,'Grades PK-12': True,
               'Grades 1-8': True,'Grades 1-6': True,
               'Grades 4-8': True,
               'Grades 6-8': True,'Grades 6-10': True, 'Grades 6-12': True,
               'Grades 7-12': True,
               'Grades 8-12': True}
grades_high = {'Grades EC-4': False, 'Grades EC-6': False,'Grades EC-12': True,
               'Grades PK-KG': False, 'Grades PK-3': False, 'Grades PK-5': False,'Grades PK-12': True,
               'Grades 1-8': False,'Grades 1-6': False,
               'Grades 4-8': False,
               'Grades 6-8': False,'Grades 6-10': True, 'Grades 6-12': True,
               'Grades 7-12': True,
               'Grades 8-12': True}
certification['cert_elem'] = certification['cert_grades'].map(grades_elem)
certification['cert_elem'] = np.where((certification.certification == False), False, certification.cert_elem) #TODO code review

certification['cert_middle'] = certification['cert_grades'].map(grades_middle)
certification['cert_middle'] = np.where((certification.certification == False), False, certification.cert_middle)

certification['cert_high'] = certification['cert_grades'].map(grades_high)
certification['cert_high'] = np.where((certification.certification == False), False, certification.cert_high)

certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject,certification,vocational,cert_elem,cert_middle,cert_high
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable,False,False,False,False,False
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education,False,False,False,False,False
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education,True,False,True,True,True


In [38]:
certification['cert_area_elem'] = np.where(certification['cert_area'] == "General Elementary (Self-Contained)", True, False)
certification['cert_area_ela'] = np.where(certification['cert_area'] == "English Language Arts", True, False)
certification['cert_area_math'] = np.where(certification['cert_area'] == "Mathematics", True, False)
certification['cert_area_sci'] = np.where(certification['cert_area'] == "Science", True, False)
certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject,certification,vocational,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable,False,False,False,False,False,False,False,False,False
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education,False,False,False,False,False,False,False,False,False
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True,False,False,False,False
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True,False,False,False,False
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education,True,False,True,True,True,False,False,False,False


In [42]:
certification = certification[['teacher_id', 'district', 'cert_level', 'cert_area', 'certification', 'vocational', 
                                 'cert_elem', 'cert_middle', 'cert_high',
                                 'cert_area_elem', 'cert_area_ela', 'cert_area_math', 'cert_area_sci']]
certification.sample(n=10)

,teacher_id,district,certification,vocational,cert_level,cert_area,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
73571,P34QVL534,115901,True,False,Special Education,Special Education,True,True,True,False,False,False,False
75416,13815*248,167901,True,False,Elementary,Mathematics,True,True,False,False,False,True,False
119780,P3131P440,015915,True,False,Elementary,General Elementary (Self-Contained),True,True,False,True,False,False,False
327928,V34*1F*43,101912,True,False,Endorsement,Bilingual Education,NaN,NaN,NaN,False,False,False,False
63364,03QVQ*548,170906,True,False,All Level,Vocational Education,True,True,True,False,False,False,False
33521,03Q021346,050902,True,False,Professional Service,Professional,True,True,True,False,False,False,False
435644,135501P34,101917,True,False,Secondary,English Language Arts,False,True,True,False,True,False,False
433130,03813*538,101917,False,False,Not Applicable,Other,False,False,False,False,False,False,False
18691,V365QV*38,026902,True,False,Supplemental,Bilingual Education,NaN,NaN,NaN,False,False,False,False
3293,13652Y147,067902,True,False,Secondary,Health and Physical Education,False,True,True,False,False,False,False


## Sort and to csv

In [54]:
certification.sort_values(by = ['teacher_id'], axis = 0)
filename = 'certs_' + year + '.csv'
certification.to_csv(os.path.join(start.data_path, 'tea', 'teachers', filename))

## Collapse to teacher level

In [60]:
teacher_cert = certification[['teacher_id', 'district', 'certification', 'vocational', 
                                 'cert_elem', 'cert_middle', 'cert_high',
                                 'cert_area_elem', 'cert_area_ela', 'cert_area_math', 'cert_area_sci']]
teacher_cert.groupby(['teacher_id', 'district']).max()
teacher_cert.sample(n = 15)

,,certification,vocational,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
teacher_id,district,,,,,,,,,
*30*0**45,101902,False,False,False,False,False,False,False,False,False
*30*0*045,057912,True,False,True,True,True,True,True,False,False
*30*00034,061902,True,False,True,True,True,True,False,True,False
*30*01145,066005,False,False,False,False,False,False,False,False,False
*30*0F443,101903,True,False,NaN,NaN,NaN,False,False,False,False
*30*0F446,108904,False,False,False,False,False,False,False,False,False
*30*0FQ45,056901,False,False,False,False,False,False,False,False,False
*30*0LP36,015910,True,False,True,True,False,True,False,False,False
*30*0V243,182903,False,False,False,False,False,False,False,False,False


## To CSV

In [65]:
filename = 'teacher_cert' + year + '.csv'
teacher_cert.to_csv(os.path.join(start.data_path, 'tea', 'teachers', filename))

# Teacher Assignments

In [43]:
pattern = "TEACHER_MASTER*.TXT"
teacher_files = []
for entry in os.listdir(teacher_datapath):
    if fnmatch.fnmatch(entry, pattern):
        teacher_files.append(entry)
teacher_files.sort()
dirs_teachers = [teacher_datapath + file for file in teacher_files]
df_list = [pd.read_csv(file, sep=",", encoding="ISO-8859-1", dtype=object) for file in dirs_teachers]
teachers = pd.concat(df_list)
teachers = teachers[teachers['ROLE NAME'] == 'TEACHER']

In [44]:
pd.set_option('display.max_columns', 100)
teachers

In [47]:
vars_to_keep = {'SCRAMBLED UNIQUE ID': 'teacher_id', 'DISTRICT NUMBER': 'district', 'DISTRICT NAME': 'distname',
                        'CAMPUS NUMBER': 'campus', 'CAMPUS NAME': 'campname',
                        'CAMPUS GRADE GROUP NAME': 'camp_grade_group',
                        'FTE': 'fte', 'ROLE FULL TIME EQUIVALENT': 'fte_teacher',
                        'SUBJECT AREA NAME 1': 'sub_area1', 'SUBJECT AREA NAME 2': 'sub_area2',
                        'SUBJECT AREA NAME 3': 'sub_area3',
                        'SUBJECT AREA NAME 4': 'sub_area4', 'SUBJECT AREA NAME 5': 'sub_area5'}
teachers = clean_tea.filter_and_rename_cols(teachers, vars_to_keep)
teachers

In [48]:
teachers

,teacher_id,district,distname,campus,campname,camp_grade_group,fte,fte_teacher,sub_area1,sub_area2,sub_area3,sub_area4,sub_area5
0,*30*5P341,031901,BROWNSVILLE ISD,031901111,SHARP EL,ELEMENTARY,1.0000,1.0002,SELF-CONTAINED,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE,SOCIAL STUDIES
1,*3034L040,031901,BROWNSVILLE ISD,031901045,OLIVEIRA MIDDLE,MIDDLE SCHOOL,1.0000,1.0000,CAREER & TECHNOLOGY EDUCATION,NaN,NaN,NaN,NaN
2,*3040V545,031901,BROWNSVILLE ISD,031901106,GARDEN PARK EL,ELEMENTARY,1.0000,0.9999,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE,SOCIAL STUDIES,PHYSICAL ED. & HEALTH
3,*3106V240,031901,BROWNSVILLE ISD,031901129,MORNINGSIDE EL,ELEMENTARY,1.0000,0.0496,OTHER,NaN,NaN,NaN,NaN
4,*31271Q42,031901,BROWNSVILLE ISD,031901003,PACE EARLY COLLEGE H S,HIGH SCHOOL,1.0000,0.6426,MATHEMATICS,SOCIAL STUDIES,PHYSICAL ED. & HEALTH,FOREIGN LANGUAGE,NaN
5,*3145*048,031901,BROWNSVILLE ISD,031901134,HUDSON EL,ELEMENTARY,1.0000,0.9996,ENGLISH LANGUAGE ARTS,MATHEMATICS,OTHER,NaN,NaN
6,*31Q1*049,031901,BROWNSVILLE ISD,031901142,PULLAM EL,ELEMENTARY,0.9465,0.9465,SELF-CONTAINED,NaN,NaN,NaN,NaN
7,*32*41148,031901,BROWNSVILLE ISD,031901103,LONGORIA EL,ELEMENTARY,1.0000,1.0000,SELF-CONTAINED,NaN,NaN,NaN,NaN
8,*3201VV42,031901,BROWNSVILLE ISD,031901033,BROWNSVILLE ACADEMIC CENTER,ELEMENTARY / SECONDARY,1.0000,1.0000,SOCIAL STUDIES,PHYSICAL ED. & HEALTH,FINE ARTS,OTHER,NaN
9,*32111123,031901,BROWNSVILLE ISD,031901007,LOPEZ EARLY COLLEGE H S,HIGH SCHOOL,0.8396,0.8400,FINE ARTS,NaN,NaN,NaN,NaN
